In [ ]:
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#from pytrr import GroTrrReader
%matplotlib inline

In [ ]:
def read_eigenvec_trr(eigenvec_trr):
    with GroTrrReader(eigenvec_trr) as trr:
        header, data = trr.read_frame(read_data=True) 
        natoms = 3 * header['natoms']
        eigvec = np.zeros((natoms, natoms))
        eigval = np.zeros((natoms, ))
        while True:
            try:
                header, data = trr.read_frame(read_data=True)
            except EOFError:
                break
            else:
                i = header["step"] - 1
                if i >= 0:
                    eigvec[i] = data['x'].flatten()                
                    eigval[i] = header["time"]
    return eigvec, eigval

eigenv = {}
for directory in glob.glob("./6peptides/gg*gg"):
    peptide = directory.split("/")[-1]
    eigenv[peptide] = read_eigenvec_trr(f"{directory}/analy/eigenvec.trr")
    
peptides = sorted(eigenv.keys())

In [ ]:
ev1, eo1 = eigenv["ggALAALAgg"]
ev2, eo2 = eigenv["ggGLYALAgg"]

n = eo1.shape[0]
mtx = np.ones((n, n))

for i in range(n):
    for j in range(n):
        mtx[i,j] = np.dot(ev1[i], ev2[j]) ** 2

fig, axs = plt.subplots(2,2, figsize=(7,6), dpi=300, sharex="col", sharey="row",
                        gridspec_kw={"hspace":0, "wspace":0, "height_ratios":[1,5], "width_ratios":[5,2]},)
ax = axs[0,0]
ax.bar(np.arange(eo2.size), eo2)
ax = axs[1,1]
ax.barh(np.arange(eo1.size), eo1)
ax = axs[1,0]
im = ax.imshow(mtx, aspect="auto", cmap="Spectral_r", vmin=0, vmax=1)

plt.colorbar(im, ax = axs[1,1])

In [ ]:
n_component = 2

mtx = np.zeros((len(peptides),) * 2)

for i, (ev1, eo1) in enumerate((eigenv[p] for p in peptides)):
    for j, (ev2, eo2) in enumerate((eigenv[p] for p in peptides)):
        
        # Coverage matrix
        mtx[i,j] = np.sqrt(np.sum(np.dot(ev1[:n_component], ev2[:n_component].T)**2))

In [ ]:
np.max(mtx)

In [ ]:
#
# -- Plot
#
fig, ax = plt.subplots(1,1, figsize=(5,4), dpi=300)
ax.grid(True, "minor", "both", color="k", linewidth=.2, alpha=1)

im = ax.imshow(mtx, cmap="jet_r", vmin=0, vmax=1)

ax.set_yticks(np.arange(14, 841, 29))
ax.set_yticks(np.arange(-.5, 840.5, 29), minor=True)
ax.set_yticklabels([p[2:5] for p in peptides[::29]], size=7)
ax.set_xticks(np.arange(14, 841, 29))
ax.set_xticks(np.arange(-.5, 840.5, 29), minor=True)
ax.set_xticklabels([p[2:5] for p in peptides[::29]], size=7, rotation=-90)

plt.colorbar(im, ax=ax, )
plt.show()

In [ ]:
for n_components in [2,3,8,10,14]:

    mtx = np.zeros((len(peptides),) * 2)

    for i, (ev1, eo1) in enumerate((eigenv[p] for p in peptides if p[2:5] == p[5:8])):
        for j, (ev2, eo2) in enumerate((eigenv[p] for p in peptides if p[2:5] == p[5:8])):

            # Proj ev2 (axis=1) on ev1 (axis=0)
            proj = np.dot(ev1, ev2.T) ** 2

            # Coverage matrix
            mtx[i,j] = np.sum(proj[:n_components] * eo2) / np.sum(eo2)

    summa = np.sum(mtx, axis=1) / len(peptides) * 29
    order = np.argsort(summa)

    print(f"{n_components} COMPONENTS:")
    print(f"  1st:\t{[p for p in peptides if p[2:5] == p[5:8]][order[-1]]} ({summa[order[-1]]:.4f})")
    print(f"  2nd:\t{[p for p in peptides if p[2:5] == p[5:8]][order[-2]]} ({summa[order[-2]]:.4f})")
    print(f"  3rd:\t{[p for p in peptides if p[2:5] == p[5:8]][order[-3]]} ({summa[order[-3]]:.4f})")
    print(f"  4th:\t{[p for p in peptides if p[2:5] == p[5:8]][order[-4]]} ({summa[order[-4]]:.4f})")
    print(f"  5th:\t{[p for p in peptides if p[2:5] == p[5:8]][order[-5]]} ({summa[order[-5]]:.4f})")
    print()

# Eigenvalue analysis

In [ ]:
filelist = sorted(glob.glob("../*peptides/gg*/analy/eigenval.xvg"))

In [ ]:
eigvalues = np.zeros((3393, 36), dtype=float)

for i, fpath in enumerate(filelist):
    arr = np.loadtxt(fpath, usecols=(1,))
    eigvalues[i,:len(arr)] = arr    

In [ ]:
explained = np.cumsum(eigvalues, axis=1)
explained = (explained.T / explained[:,-1]).T

In [ ]:
q25, q50, q75 = np.quantile(explained, [.25, .5, .75], axis=0)

fig, ax = plt.subplots()

ax.fill_between(np.arange(1, q50.size+1), q25, q75, alpha=.3)
ax.plot(np.arange(1,q50.size+1), q50)

In [ ]:
q50[4]